In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [6]:
def process_sequences():
    Entrez.email = 'katyachistiakova.05@mail.ru'  
    input_file = r"C:\Users\2slon\OneDrive\Рабочий стол\5sem\pestiviruses\full_sequences.acc"

    with open(input_file, 'r') as ac:
        accessions = [line.strip() for line in ac if line.strip()]
    
    total_count = len(accessions)
    print(f"Всего accession numbers в файле: {total_count}")

    success_count = 0
    no_cds_count = 0
    error_count = 0

    out_cds = open('CDS_full_sequences.fasta', 'w')
    out_no_cds = open('no_cds_sequences.fasta', 'w')
    out_log = open('no_cds.log', 'w')

    for accession in accessions:
        try:
            handle = Entrez.efetch(db="nucleotide", id=accession, rettype='gbwithparts', retmode='text')
            record = SeqIO.read(handle, "genbank")
            handle.close()
            cds_features = [feat for feat in record.features if feat.type == "CDS"]

            if cds_features:
                CDS = cds_features[0].extract(record.seq)
                cds_record = SeqRecord(
                    CDS,
                    id=record.id,
                    name=record.name,
                    description=f"{record.description}"
                )
                SeqIO.write(cds_record, out_cds, 'fasta')
                success_count += 1

            else:
                full_record = SeqRecord(
                    record.seq,
                    id=record.id,
                    name=record.name,
                    description=f"{record.description}"
                )
                SeqIO.write(full_record, out_no_cds, 'fasta')
                out_log.write(f"{accession}\n")
                no_cds_count += 1

        except Exception as e:
            error_count += 1
            out_log.write(f"ERROR: {accession} - {str(e)}\n")

    out_cds.close()
    out_no_cds.close()
    out_log.close()

    
    print(f"Total count: {total_count}")
    print(f"Success: {success_count}")
    print(f"No CDS: {no_cds_count}")
    print(f"Errors: {error_count}")
   
process_sequences()


Всего accession numbers в файле: 1857
Total count: 1857
Success1788
No CDS: 69
Errors: 0
